Below is code that reads for-notebook.csv created by Clojure converter and trains a neural network for classifying next day stock / index behavior (0 = bear, 1 = bull).

In [ ]:
import tensorflow as tf
import numpy as np
import sklearn as skl
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

desired_accuracy = 0.95

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc') > desired_accuracy):
      print("\nReached desired accuracy so cancelling training!")
      self.model.stop_training = True

dataset=np.loadtxt("for-notebook.csv",delimiter=",")
x_column_count = dataset.shape[1] - 1;
x=dataset[:,0:x_column_count]
y=dataset[:,x_column_count]

min_max_scaler = skl.preprocessing.MinMaxScaler()
x_scale = min_max_scaler.fit_transform(x)

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(x_scale, y, test_size=0.3)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=0.5)

print("Data shapes: ", x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape)

callbacks = myCallback()

# Feel free to comment out kernel regularizers and Dropout layers.
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(x_column_count,)
                         , kernel_regularizer=tf.keras.regularizers.l2(0.01)
                         ),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu', input_shape=(x_column_count,)
                         , kernel_regularizer=tf.keras.regularizers.l2(0.01)
                         ),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid'
                         , kernel_regularizer=tf.keras.regularizers.l2(0.01)
                         )
])


# Adam and sgd are good optimizers to try.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
hist = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_val, y_val), callbacks=[callbacks])
evaluation = model.evaluate(x_test, y_test)
print("Loss: ", evaluation[0], ", Accuracy: ", evaluation[1])

Next let's visualize the training and validation losses.

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()


Then let's visualize the training and validation accuracies.

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()
